##Pyspark Analysis Project on Airline Dataset

In [1]:
pip install pyspark  #installing pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 46.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=41ff4b61ff00110b534de7b5eea02588189e4f4cefcbb5c6d55ec50522c95cba
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
# initializing pyspark libraries 
from pyspark.sql import SparkSession     # an entry point to PySpark          

spark = SparkSession.builder.appName('Assignment').getOrCreate()   # i.e creation of spark session 

In [3]:
df = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/content/Airline_data.csv")  # create dataframe from csv file

### Q1 Show a sample of 5 records from dataset.

In [4]:
df.show(5)  # first five records

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1552|           UA|      183

### Q2. Read the data with data types.

In [5]:
df.printSchema()  # use for showing datatypes

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |

get datatypes of each variable(column) in dataset

### OR

In [6]:
df.dtypes  # # use for showing datatypes

[('Year', 'int'),
 ('Month', 'int'),
 ('DayofMonth', 'int'),
 ('DayOfWeek', 'int'),
 ('DepTime', 'int'),
 ('CRSDepTime', 'int'),
 ('ArrTime', 'int'),
 ('CRSArrTime', 'int'),
 ('UniqueCarrier', 'string'),
 ('FlightNum', 'int'),
 ('TailNum', 'string'),
 ('ActualElapsedTime', 'int'),
 ('CRSElapsedTime', 'int'),
 ('AirTime', 'string'),
 ('ArrDelay', 'int'),
 ('DepDelay', 'int'),
 ('Origin', 'string'),
 ('Dest', 'string'),
 ('Distance', 'int'),
 ('TaxiIn', 'string'),
 ('TaxiOut', 'string'),
 ('Cancelled', 'int'),
 ('CancellationCode', 'string'),
 ('Diverted', 'int'),
 ('CarrierDelay', 'string'),
 ('WeatherDelay', 'string'),
 ('NASDelay', 'string'),
 ('SecurityDelay', 'string'),
 ('LateAircraftDelay', 'string')]

here get datatypes of variables(columns in dataset

### Q3. Make a new column MonthStr, Which has months in form of 01, 02, 03, ..., 12.

In [8]:
# importing the required libraries
from pyspark.sql.functions import udf,col   # inport udf 
from pyspark.sql.types import StringType     # import strung type


d = {}  # empty list
# Changing the data type of 'Month' column to string
df1 = df.withColumn("Month", df["Month"].cast(StringType()))
for i in df1.collect():
  d[i['Month']] = '0'+i['Month']       # use foor loop to make string to all column

def monthstr(Month):          # use functio
  return str(d.get(Month))
# creating the udf
a = udf(monthstr)
df1 = df1.withColumn("MonthStr",a(col('Month')))   # passing the Month column
df1.show(10)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|MonthStr|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1

### Q4. Find the # of flights each airline made.

In [9]:
df.groupBy("UniqueCarrier").agg({'FlightNum':'count'}).show()  # use of groupby and aggregate function


+-------------+----------------+
|UniqueCarrier|count(FlightNum)|
+-------------+----------------+
|           UA|             426|
+-------------+----------------+



From above table get number of flights through unique carrieer

In [10]:
df.groupBy("Origin").agg({'FlightNum':'count'}).show()  # use of gruopby and aggregate function


+------+----------------+
|Origin|count(FlightNum)|
+------+----------------+
|   LIH|              30|
|   HNL|             124|
|   EWR|              24|
|   DEN|              30|
|   IAD|              30|
|   SFO|             130|
|   PHL|              29|
|   OGG|              29|
+------+----------------+



From above table get number of flights through origin(airport)




### Q5. Find the mean Arrival delay delay per origination airport





In [11]:
from pyspark.sql import functions as F          # import function
df.groupBy("Origin").agg(F.mean('ArrDelay')).show() # use of gruopby and aggregate function for mean

+------+-------------------+
|Origin|      avg(ArrDelay)|
+------+-------------------+
|   LIH|0.16666666666666666|
|   HNL|  14.21774193548387|
|   EWR|               9.25|
|   DEN| 20.166666666666668|
|   IAD| 12.966666666666667|
|   SFO| 11.215384615384615|
|   PHL|  6.827586206896552|
|   OGG|  16.24137931034483|
+------+-------------------+



From above table get mean Arrival delay delay per origination airport

### OR

In [12]:
df.groupBy("Origin").agg({'ArrDelay':'avg'}).show()  # use of gruopby and aggregate function for mean

+------+-------------------+
|Origin|      avg(ArrDelay)|
+------+-------------------+
|   LIH|0.16666666666666666|
|   HNL|  14.21774193548387|
|   EWR|               9.25|
|   DEN| 20.166666666666668|
|   IAD| 12.966666666666667|
|   SFO| 11.215384615384615|
|   PHL|  6.827586206896552|
|   OGG|  16.24137931034483|
+------+-------------------+



From above table get mean Arrival delay delay per origination airp

### Q6 What is the average departure delay from each airport?

In [13]:
df.groupBy("Origin").agg({'DepDelay':'avg'}).show()  # use of gruopby and aggregate function for average

+------+-------------------+
|Origin|      avg(DepDelay)|
+------+-------------------+
|   LIH|-3.7666666666666666|
|   HNL|  3.217741935483871|
|   EWR|  4.958333333333333|
|   DEN|               27.6|
|   IAD|                8.9|
|   SFO| 19.646153846153847|
|   PHL| 16.137931034482758|
|   OGG|                6.0|
+------+-------------------+



from Above table average departure delay from each airport